In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import Discrete, Box
import numpy as np
import scipy.stats as stats
import random
import time
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from CustomDictObsEnv import CustomDictObsEnv, feaures_model_coefficient

In [3]:
ray.init(ignore_reinit_error=True, num_cpus=2)

2024-03-13 14:37:17,967	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 


Python version:,3.10.13
Ray version:,2.9.3
Dashboard:,http://127.0.0.1:8267


In [4]:
from environments.simulator_env_1 import SimulatorEnv1, LogisticModelFeature
from utils import FEATURES
INTERCEPT = 10
N = 10000
simulator_env = SimulatorEnv1(
    discrete_action_space=True,
    intercept=INTERCEPT,
    features=FEATURES,
    n_reps=N,
    first_arm=0.1,
    last_arm=0.9,)
input_data = simulator_env.get_obs_dataset(100)
input_data.head()

,apw,base_price,customer_type,hotel_level,night,occupancy,promotion
0,6,186.449962,1,1,8,2,3
1,72,199.390025,1,3,5,5,1
2,26,159.450858,1,4,6,2,0
3,0,175.077632,0,0,5,5,3
4,11,172.820999,1,4,3,2,3


In [5]:
input_data_copy = input_data.copy()

In the original simulation environment, the observation spaces are represented using a dictionary structure, where each key is associated with a custom type. This approach, however, encounters compatibility issues with Ray, prompting a reevaluation of the data representation strategy.

The custom type, initially a single entity, not only facilitated the generation of sample data based on mathematical and logical distributions within the environment class but also encapsulated feature coefficients crucial for the simulation's fidelity.

To address the compatibility concerns with Ray and enhance the system's modularity, the custom type has been decomposed into three distinct components: `input_data`, `features_model_coefficient`, and a method named `add_data()`. This restructuring aims to preserve the original functionality while improving integration with Ray's framework .


In [ ]:
config = PPOConfig()\
        .environment(env=CustomDictObsEnv, 
                    env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
                     disable_env_checking=True)\
        .rollouts(num_rollout_workers=1)\
        .framework("tf2") \
        .training(kl_coeff=0.0, entropy_coeff=0.1)  # Adjust these values based on your needs       

algo= config.build()

for i in range(5):
    results = algo.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/sit

Iter: 0; avg. reward=3525.5161747591746
Iter: 1; avg. reward=3560.812018633732
Iter: 2; avg. reward=3567.480581244634
Iter: 3; avg. reward=3572.387837787608
Iter: 4; avg. reward=3583.5967832463184


In [6]:
import ray

from ray.rllib.algorithms.random_agent.random_agent import RandomAgentConfig, RandomAgent
from ray.tune.registry import register_env

In [8]:
random_config = RandomAgentConfig()\
        .environment(env=CustomDictObsEnv, 
                    env_config={
                        "intercept": 10.0,
                        "n_reps": 100,
                        "fetures_model_coefficient": feaures_model_coefficient,
                        "data": input_data_copy,},
                     disable_env_checking=True)\
        .rollouts(num_rollout_workers=1)\
        .framework("tf2") \
        .training()  # Adjust these values based on your needs       

random_algo= random_config.build()

for i in range(5):
    results = random_algo.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")
ray.shutdown()


2024-03-13 14:37:43,551	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/random_agent/` has been deprecated. This algorithm will be removed by ray 2.9 It is being moved to the ray/rllib_contrib dir. See https://github.com/ray-project/enhancements/blob/main/reps/2023-04-28-remove-algorithms-from-rllib.md for more details. Any associated components (e.g. policies) will also be moved. This will raise an error in the future!
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/xizhang/.pyenv/versions/streamlit-my-app/envs/3.11.4/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: 

Iter: 0; avg. reward=3445.1731270651385
Iter: 1; avg. reward=3434.288981589058
Iter: 2; avg. reward=3411.027238230108
Iter: 3; avg. reward=3343.6841726880134
Iter: 4; avg. reward=3386.760274037462
